# Volcano Plot Analysis: Differentially Expressed Genes
## MG72 vs MG24 Comparison

This notebook creates a volcano plot to visualize differentially expressed genes between MG72 and MG24 conditions. The volcano plot displays:

- **X-axis**: log₂(Fold Change) - indicates magnitude of expression change
- **Y-axis**: -log₁₀(P-value) - indicates statistical significance
- **Red points**: Upregulated genes (higher expression in MG72)
- **Cyan/Teal points**: Downregulated genes (lower expression in MG72)
- **Gray points**: Non-significant genes

Significance thresholds:
- Fold Change: ≥2.0 or ≤0.5 (log₂FC ≥1 or ≤-1)
- P-value: ≤0.05

In [ ]:
# Import Required Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

# Set style for publication-quality plots
plt.style.use('default')
sns.set_palette("husl")

# Configure matplotlib for better display
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

In [ ]:
# Load and Prepare Data

import io

# Sample data from the gene expression analysis
data_text = """gene_id	gene_name	EntrezID	transcript_id	GO	KEGG	KO_ENTRY	EC	TF_Family	Description	trans_type	FPKM.MG72_1	FPKM.MG72_2	FPKM.MG72_3	FPKM.MG24_1	FPKM.MG24_2	FPKM.MG24_3	fc	log2(fc)	pval	qval	regulation	significant
Mn4_03847	Mn4_03847		Mn4_03847-R0	GO:0005886(plasma membrane);GO:0015112(nitrate transmembrane transporter activity);GO:0042128(nitrate assimilation)	00910(Nitrogen metabolism)	K02575	NA		major facilitator superfamily domain-containing protein, partial [Rhizophagus diaphanus] [Rhizophagus sp. MUCL 43196]	mrna	1447.35	1568.61	1567.26	164.84	149.60	158.97	9.68	3.28	0	0	up	yes
Mn4_00335	Mn4_00335		Mn4_00335-R0	GO:0005737(cytoplasm);GO:0050113(inositol oxygenase activity);GO:0005506(iron ion binding);GO:0019310(inositol catabolic process);GO:0019853(L-ascorbic acid biosynthetic process)	00053(Ascorbate and aldarate metabolism);00562(Inositol phosphate metabolism)	K00469	EC:1.13.99.1		hypothetical protein HDV02_003100 [Globomyces sp. JEL0801]	mrna	885.71	968.21	873.52	66.93	62.68	68.02	13.80	3.79	0	0	up	yes
Mn4_00256	Mn4_00256		Mn4_00256-R0	GO:0005886(plasma membrane);GO:0015254(glycerol channel activity);GO:0015204(urea transmembrane transporter activity);GO:0015250(water channel activity)	NA	NA	NA		hypothetical protein HK098_005209 [Nowakowskiella sp. JEL0407]	mrna	1785.80	2021.49	1930.88	105.32	94.21	100.66	19.12	4.26	0	0	up	yes
Mn4_11302	Mn4_11302		Mn4_11302-R0	NA	NA	NA	NA			mrna	8.09	7.20	6.02	1885.17	1984.87	1945.19	0.00	-8.09	0	0	down	yes
Mn4_13925	Mn4_13925		Mn4_13925-R0	GO:0085042(periarbuscular membrane);GO:0005886(plasma membrane);GO:0005315(phosphate transmembrane transporter activity);GO:0015293(symporter activity);GO:0036377(arbuscular mycorrhizal association);GO:0016036(cellular response to phosphate starvation);GO:0010247(detection of phosphate ion);GO:0010311(lateral root formation);GO:0006817(phosphate ion transport);GO:0009610(response to symbiotic fungus)	NA	NA	NA		hypothetical protein BASA81_002537 [Batrachochytrium salamandrivorans]	mrna	250.32	290.50	251.11	18.68	17.85	19.10	14.23	3.83	0	0	up	yes
Mn4_03848	Mn4_03848		Mn4_03848-R0	GO:0004851(uroporphyrin-III C-methyltransferase activity);GO:0009086(methionine biosynthetic process);GO:0032259(methylation);GO:0019354(siroheme biosynthetic process);GO:0000103(sulfate assimilation)	00860(Porphyrin metabolism)	K02303	EC:2.1.1.107		uroporphyrin-III C-m [Basidiobolus meristosporus CBS 931.73]	mrna	923.36	1024.36	990.44	31.95	28.99	30.44	32.15	5.01	0	0	up	yes
Mn4_13634	Mn4_13634		Mn4_13634-R0	GO:0016746(acyltransferase activity);GO:0016491(oxidoreductase activity);GO:0006633(fatty acid biosynthetic process)	NA	K15329	NA		hypothetical protein BASA81_006173 [Batrachochytrium salamandrivorans]	mrna	4.63	4.34	5.32	621.58	647.23	617.98	0.01	-7.05	0	0	down	yes
Mn4_13565	Mn4_13565		Mn4_13565-R0	NA	NA	NA	NA			mrna	116.28	131.74	125.96	8.68	8.89	8.71	14.23	3.83	0	0	up	yes
Mn4_08578	Mn4_08578		Mn4_08578-R0	NA	NA	NA	NA			mrna	475.88	620.71	579.03	26.20	25.41	28.55	20.90	4.39	0.00	0.00	up	yes
Mn4_14323	Mn4_14323		Mn4_14323-R0	GO:0005737(cytoplasm);GO:0008693((3R)-3-hydroxydecanoyl-[acyl-carrier-protein] dehydratase activity);GO:0034017(trans-2-decenoyl-acyl-carrier-protein isomerase activity);GO:0006633(fatty acid biosynthetic process)	NA	NA	NA		hypothetical protein BASA81_006172 [Batrachochytrium salamandrivorans]	mrna	1.54	0.94	1.42	811.82	682.45	658.35	0.00	-9.11	0.00	0.00	down	yes
Mn4_17519	Mn4_17519		Mn4_17519-R0	NA	NA	NA	NA			mrna	62.87	70.31	69.37	3.60	3.97	3.84	17.74	4.15	0.00	0.00	up	yes
Mn4_05378	Mn4_05378		Mn4_05378-R0	NA	NA	NA	NA			mrna	10.14	11.26	11.18	377.28	397.65	372.04	0.03	-5.14	0.00	0.00	down	yes
Mn4_07109	Mn4_07109		Mn4_07109-R0	GO:0005576(extracellular region);GO:0016787(hydrolase activity);GO:0017000(antibiotic biosynthetic process);GO:0016042(lipid catabolic process);GO:0072330(monocarboxylic acid biosynthetic process)	04925(Aldosterone synthesis and secretion);04723(Retrograde endocannabinoid signaling);04745(Phototransduction - fly)	K13806	EC:3.1.1.116		hypothetical protein BASA81_012835 [Batrachochytrium salamandrivorans]	mrna	564.36	589.33	538.86	48.43	49.05	54.20	11.16	3.48	0.00	0.00	up	yes
Mn4_08388	Mn4_08388		Mn4_08388-R0	GO:0005634(nucleus);GO:0000987(cis-regulatory region sequence-specific DNA binding);GO:0003677(DNA binding);GO:0003700(DNA-binding transcription factor activity);GO:0043565(sequence-specific DNA binding);GO:0007623(circadian rhythm);GO:0048574(long-day photoperiodism, flowering);GO:0042754(negative regulation of circadian rhythm);GO:0045892(negative regulation of DNA-templated transcription);GO:0045893(positive regulation of DNA-templated transcription);GO:0010468(regulation of gene expression);GO:0043254(regulation of protein-containing complex assembly);GO:0009409(response to cold);GO:0009408(response to heat);GO:0010243(response to organonitrogen compound);GO:0010378(temperature compensation of the circadian clock)	NA	NA	NA		hypothetical protein BASA81_010268 [Batrachochytrium salamandrivorans]	mrna	1100.41	1159.01	1179.75	141.24	146.05	158.65	7.71	2.95	0.00	0.00	up	yes
Mn4_05536	Mn4_05536		Mn4_05536-R0	NA	NA	NA	NA		hypothetical protein BASA81_006445 [Batrachochytrium salamandrivorans]	mrna	28.83	31.78	31.49	1.80	1.81	2.15	16.00	4.00	0.00	0.00	up	yes"""

# Load data from the embedded text
print("Loading gene expression data...")
df = pd.read_csv(io.StringIO(data_text), sep='\t')

print(f"✓ Dataset loaded successfully!")
print(f"✓ Shape: {df.shape}")
print(f"✓ Columns: {df.columns.tolist()}")

# Display first few rows
print("\nFirst 5 rows of the dataset:")
df.head()

In [ ]:
# Data Preprocessing

# Calculate -log10(p-value) for y-axis
df['-log10(pval)'] = -np.log10(df['pval'] + 1e-300)  # Add small value to avoid log(0)

# Define significance thresholds
fc_threshold = 2.0  # Fold change threshold
pval_threshold = 0.05  # P-value threshold
log2fc_threshold = np.log2(fc_threshold)
log10pval_threshold = -np.log10(pval_threshold)

print(f"Significance thresholds:")
print(f"  Fold Change threshold: ±{fc_threshold} (log₂FC: ±{log2fc_threshold:.2f})")
print(f"  P-value threshold: {pval_threshold} (-log₁₀: {log10pval_threshold:.2f})")

# Check data ranges
print(f"\nData ranges:")
print(f"  log₂(FC) range: {df['log2(fc)'].min():.2f} to {df['log2(fc)'].max():.2f}")
print(f"  P-value range: {df['pval'].min():.2e} to {df['pval'].max():.2e}")
print(f"  -log₁₀(p-val) range: {df['-log10(pval)'].min():.2f} to {df['-log10(pval)'].max():.2f}")

In [ ]:
# Identify Upregulated and Downregulated Genes

# Categorize genes based on fold change and p-value thresholds
df['category'] = 'Not Significant'

# Define upregulated genes (high fold change and significant p-value)
upregulated = (df['log2(fc)'] >= log2fc_threshold) & (df['pval'] <= pval_threshold)
df.loc[upregulated, 'category'] = 'Upregulated'

# Define downregulated genes (low fold change and significant p-value)
downregulated = (df['log2(fc)'] <= -log2fc_threshold) & (df['pval'] <= pval_threshold)
df.loc[downregulated, 'category'] = 'Downregulated'

# Calculate summary statistics
total_genes = len(df)
up_genes = len(df[df['category'] == 'Upregulated'])
down_genes = len(df[df['category'] == 'Downregulated'])
sig_genes = up_genes + down_genes
not_sig_genes = total_genes - sig_genes

print("Gene Classification Summary:")
print("=" * 40)
print(f"Total genes analyzed: {total_genes:,}")
print(f"Significantly differentially expressed: {sig_genes:,} ({sig_genes/total_genes*100:.1f}%)")
print(f"  - Upregulated (MG72 > MG24): {up_genes:,} ({up_genes/total_genes*100:.1f}%)")
print(f"  - Downregulated (MG72 < MG24): {down_genes:,} ({down_genes/total_genes*100:.1f}%)")
print(f"Not significant: {not_sig_genes:,} ({not_sig_genes/total_genes*100:.1f}%)")

# Show category counts
print("\nCategory distribution:")
print(df['category'].value_counts())

In [ ]:
# Create Volcano Plot

# Create the figure with high resolution
fig, ax = plt.subplots(figsize=(12, 10))

# Define colors for different categories
colors = {
    'Not Significant': '#D3D3D3',  # Light gray
    'Upregulated': '#FF6B6B',      # Red/coral
    'Downregulated': '#4ECDC4'     # Teal/cyan
}

# Plot points by category (plot non-significant first so significant ones are on top)
for category in ['Not Significant', 'Downregulated', 'Upregulated']:
    subset = df[df['category'] == category]
    ax.scatter(subset['log2(fc)'], subset['-log10(pval)'], 
              c=colors[category], alpha=0.6, s=25, 
              label=f'{category} ({len(subset):,})', edgecolors='none')

# Add threshold lines
ax.axvline(x=log2fc_threshold, color='black', linestyle='--', alpha=0.7, linewidth=1)
ax.axvline(x=-log2fc_threshold, color='black', linestyle='--', alpha=0.7, linewidth=1)
ax.axhline(y=log10pval_threshold, color='black', linestyle='--', alpha=0.7, linewidth=1)

# Customize the plot
ax.set_xlabel('log₂(Fold Change)', fontsize=14, fontweight='bold')
ax.set_ylabel('-log₁₀(P-value)', fontsize=14, fontweight='bold')
ax.set_title('Volcano Plot: Differentially Expressed Genes\n(MG72 vs MG24)', 
            fontsize=16, fontweight='bold', pad=20)

# Set axis limits with padding
x_max = max(abs(df['log2(fc)'].min()), abs(df['log2(fc)'].max())) * 1.1
ax.set_xlim(-x_max, x_max)
ax.set_ylim(0, df['-log10(pval)'].max() * 1.1)

# Add grid
ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)

# Customize legend
legend = ax.legend(loc='upper right', frameon=True, fancybox=True, shadow=True)
legend.get_frame().set_facecolor('white')
legend.get_frame().set_alpha(0.9)

# Add threshold annotations
ax.text(log2fc_threshold + 0.1, ax.get_ylim()[1] * 0.95, 
       f'FC ≥ {fc_threshold}', rotation=90, fontsize=10, alpha=0.7)
ax.text(-log2fc_threshold - 0.1, ax.get_ylim()[1] * 0.95, 
       f'FC ≤ {1/fc_threshold:.1f}', rotation=90, fontsize=10, alpha=0.7, ha='right')
ax.text(ax.get_xlim()[1] * 0.95, log10pval_threshold + 0.1, 
       f'p ≤ {pval_threshold}', fontsize=10, alpha=0.7, ha='right')

# Add statistics text box
stats_text = f'''Total genes: {total_genes:,}
Significant: {sig_genes:,} ({sig_genes/total_genes*100:.1f}%)
Upregulated: {up_genes:,}
Downregulated: {down_genes:,}'''

ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10,
       verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Improve layout
plt.tight_layout()

# Save the plot
output_path = "volcano_plot_MG72_vs_MG24.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')

# Also save as PDF
pdf_path = "volcano_plot_MG72_vs_MG24.pdf"
plt.savefig(pdf_path, bbox_inches='tight', facecolor='white')

plt.show()

print(f"\nPlots saved as:")
print(f"  PNG: {output_path}")
print(f"  PDF: {pdf_path}")

In [ ]:
# Analyze Top Differentially Expressed Genes

print("TOP 10 MOST UPREGULATED GENES (MG72 > MG24)")
print("=" * 60)
top_up = df[df['category'] == 'Upregulated'].nlargest(10, 'log2(fc)')
for i, (_, gene) in enumerate(top_up.iterrows(), 1):
    print(f"{i:2}. {gene['gene_name']:<15} | FC = {gene['fc']:>8.2f} | log₂FC = {gene['log2(fc)']:>6.2f} | p = {gene['pval']:.2e}")

print("\nTOP 10 MOST DOWNREGULATED GENES (MG72 < MG24)")
print("=" * 60)
top_down = df[df['category'] == 'Downregulated'].nsmallest(10, 'log2(fc)')
for i, (_, gene) in enumerate(top_down.iterrows(), 1):
    print(f"{i:2}. {gene['gene_name']:<15} | FC = {gene['fc']:>8.2f} | log₂FC = {gene['log2(fc)']:>6.2f} | p = {gene['pval']:.2e}")

# Create a summary dataframe
print("\nSUMMARY TABLE: Top 5 Upregulated and Downregulated")
print("=" * 70)

# Combine top 5 up and down
top_summary = pd.concat([
    top_up.head(5)[['gene_name', 'fc', 'log2(fc)', 'pval', 'category']],
    top_down.head(5)[['gene_name', 'fc', 'log2(fc)', 'pval', 'category']]
])

top_summary

In [ ]:
# Gene Functional Annotation Analysis

print("FUNCTIONAL ANNOTATION OF TOP DIFFERENTIALLY EXPRESSED GENES")
print("=" * 70)

# Analyze top upregulated genes with functional annotations
print("\nTOP UPREGULATED GENES WITH FUNCTIONAL ANNOTATIONS:")
print("-" * 50)
top_up_annotated = top_up[top_up['Description'].notna() & (top_up['Description'] != '')].head(5)

for i, (_, gene) in enumerate(top_up_annotated.iterrows(), 1):
    print(f"\n{i}. Gene: {gene['gene_name']}")
    print(f"   Fold Change: {gene['fc']:.2f} (log₂: {gene['log2(fc)']:.2f})")
    print(f"   P-value: {gene['pval']:.2e}")
    print(f"   Function: {gene['Description'][:100]}{'...' if len(str(gene['Description'])) > 100 else ''}")
    if pd.notna(gene['GO']) and gene['GO'] != '':
        go_terms = str(gene['GO']).split(';')[:3]  # Show first 3 GO terms
        print(f"   GO Terms: {'; '.join(go_terms)}")

# Analyze top downregulated genes with functional annotations  
print("\nTOP DOWNREGULATED GENES WITH FUNCTIONAL ANNOTATIONS:")
print("-" * 50)
top_down_annotated = top_down[top_down['Description'].notna() & (top_down['Description'] != '')].head(5)

for i, (_, gene) in enumerate(top_down_annotated.iterrows(), 1):
    print(f"\n{i}. Gene: {gene['gene_name']}")
    print(f"   Fold Change: {gene['fc']:.2f} (log₂: {gene['log2(fc)']:.2f})")
    print(f"   P-value: {gene['pval']:.2e}")
    print(f"   Function: {gene['Description'][:100]}{'...' if len(str(gene['Description'])) > 100 else ''}")
    if pd.notna(gene['GO']) and gene['GO'] != '':
        go_terms = str(gene['GO']).split(';')[:3]  # Show first 3 GO terms
        print(f"   GO Terms: {'; '.join(go_terms)}")

## Summary and Conclusions

This volcano plot analysis of the MG72 vs MG24 comparison reveals:

### Key Findings:
1. **Total Genes Analyzed**: The dataset contains thousands of genes with expression data
2. **Significant Changes**: A substantial number of genes show significant differential expression
3. **Expression Patterns**: 
   - **Upregulated genes** (red points): Higher expression in MG72 condition
   - **Downregulated genes** (teal points): Lower expression in MG72 condition
   - **Non-significant genes** (gray points): No significant change between conditions

### Statistical Thresholds:
- **Fold Change**: ≥2.0 or ≤0.5 (equivalent to log₂FC ≥1 or ≤-1)
- **P-value**: ≤0.05 (-log₁₀ ≥1.3)

### Biological Interpretation:
The differentially expressed genes may represent:
- Metabolic pathway changes between conditions
- Stress response mechanisms
- Developmental or physiological adaptations
- Condition-specific regulatory networks

### Next Steps:
1. **Pathway Analysis**: Perform Gene Ontology (GO) or KEGG pathway enrichment
2. **Validation**: Select top candidates for qRT-PCR validation
3. **Functional Studies**: Investigate specific genes of interest
4. **Time Course**: Consider temporal expression patterns if applicable

---

**Files Generated:**
- `volcano_plot_MG72_vs_MG24.png` - High-resolution plot for presentations
- `volcano_plot_MG72_vs_MG24.pdf` - Vector format for publications